In [5]:
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Metal
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


In [6]:
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
# torch.backends.mps.enable_flash_sdp(False)


In [4]:
from diffusers import StableDiffusionPipeline
from transformers import AutoTokenizer
import os
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, DDPMScheduler, LMSDiscreteScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import clip
from tqdm import tqdm
from scripts import eng_processor
from scripts.text_preprocessing import TextImageDataset
from scripts.train_model import BertEncoder, Generator, Discriminator
from scripts.inference import generate_image_from_text
from scripts.utils import combine_dataset, Evaluator
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from torchvision import transforms


/opt/anaconda3/envs/intro_dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
combined_df = pd.read_csv("datasets/combined_dataset.csv")
combined_df.tail(5)

,image_path,caption,source
90483,datasets/coco_dataset/train2017/000000370808.jpg,A black cat looking out the window at a black ...,MSCOCO
90484,datasets/coco_dataset/train2017/000000370808.jpg,Black cat sitting on window ledge looking outs...,MSCOCO
90485,datasets/coco_dataset/train2017/000000370808.jpg,A black cat looks out the window as a crow out...,MSCOCO
90486,datasets/coco_dataset/train2017/000000370808.jpg,A cat by a window with a small bird outside.,MSCOCO
90487,datasets/coco_dataset/train2017/000000370808.jpg,A cat watches a bird through a window.,MSCOCO


In [36]:
# combined_df= combined_df.where(combined_df["source"]=="Flickr")

# combined_df = combined_df.dropna()
combined_df = combined_df.sample(frac=1, random_state=42).head(10000)
combined_df = combined_df.reset_index(drop=True)  # Reset index to sequential integers

for i in range(len(combined_df)):
    combined_df.caption[i] = eng_processor.main(combined_df.caption[i])

/var/folders/9l/8k8d_tn97jdgsjynhl5q7yr80000gn/T/ipykernel_98781/1002816691.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_df.caption[i] = eng_processor.main(combined_df.caption[i])


In [37]:
image_to_captions = defaultdict(list)

for _, row in combined_df.iterrows():
    img_path = row['image_path']
    image_to_captions[img_path].append(row['caption'])

In [38]:
image_paths = list(image_to_captions.keys())
train_ids, test_ids = train_test_split(image_paths, test_size=0.2, random_state=42)


In [39]:
# image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Lambda(lambda x: (x * 2) - 1)  # Scale from [0,1] to [-1,1]
])


train_dataset = TextImageDataset(
    image_to_captions=image_to_captions,
    image_paths=train_ids,  
    transform=transform
)

test_dataset = TextImageDataset(
    image_to_captions=image_to_captions,
    image_paths=test_ids,  
    transform=transform
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

Dataset size: 7988 (all captions included)
Dataset size: 2011 (all captions included)


In [13]:
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
clip_text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").to(device)



In [14]:
clip_text_encoder.eval()


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), ep

In [16]:
# Load the pretrained pipeline
pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",  # or any other model
    torch_dtype=torch.float16,         # Use float16 only if GPU/MLCompute is supported
)

pipeline.to("mps")  # Use M1/M2 GPU via Metal backend

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|█████████████| 7/7 [00:07<00:00,  1.02s/it]


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.33.1",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [8]:
class Projection(nn.Module):
    def __init__(self, in_dim=768, out_dim=768):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.proj(x)


In [16]:

projection = Projection().to(device)

# === 4. Stable Diffusion Decoder ===
vae = pipeline.vae.to(device)
unet = pipeline.unet.to(device)
scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)



In [17]:
# Freeze everything except projection
for param in vae.parameters(): param.requires_grad = False
for param in unet.parameters(): param.requires_grad = False

optimizer = torch.optim.Adam(projection.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()


In [18]:
for epoch in range(10):

    unet.train()
    clip_text_encoder.eval()
    vae.eval()
    projection.train()


    for i, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
        images = batch["image"].to(device)
        texts = batch["caption_text"]
        
        # Match input dtype with model
        dtype = next(vae.parameters()).dtype
        images = images.to(dtype=dtype)

        with torch.no_grad():
            # Text encoding
            inputs = clip_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
            inputs = {key: val.to(device) for key, val in inputs.items()}

            text_feats = clip_text_encoder(**inputs).last_hidden_state  # (B, seq_len, 768)


            # get latent for vae
            latents = vae.encode(images).latent_dist.sample() * 0.18215


        pooled_feats = text_feats.mean(dim=1)  # (B, 768)
        projected_feats = projection(pooled_feats)

        # add noise to it
        with torch.no_grad():

            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, 1000, (latents.shape[0],), device=device).long()
            noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        projected_feats = projected_feats.to(dtype=noisy_latents.dtype).unsqueeze(1)  # (B, 1, D)

        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=projected_feats).sample

        loss = loss_fn(noise_pred, noise)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if i % 100 == 0:
            print(f"[Batch {i}/{len(train_loader)}] [Loss: {loss.item():.4f}]")

        # Clear memory
        del images, texts, inputs, text_feats, pooled_feats, projected_feats
        del latents, noise, timesteps, noisy_latents, noise_pred
        torch.mps.empty_cache()

    print(f"Epoch {epoch} finished")


# Save model after training
torch.save({
    'unet': unet.state_dict(),
    'clip_text_encoder': clip_text_encoder.state_dict(),
    'projection': projection.state_dict()
}, "final_model_checkpoint.pt")

print("Training completed ✅")

Training Epoch 1:   0%|                      | 1/4016 [00:01<1:23:43,  1.25s/it]

[Batch 0/4016] [Loss: 0.3232]


Training Epoch 1:   3%|▌                     | 101/4016 [00:40<25:30,  2.56it/s]

[Batch 100/4016] [Loss: 0.3823]


Training Epoch 1:   5%|█                     | 201/4016 [01:19<25:03,  2.54it/s]

...

[Batch 3900/4016] [Loss: 0.4487]


Training Epoch 10: 100%|███████████████████▉| 4001/4016 [26:39<00:05,  2.59it/s]

[Batch 4000/4016] [Loss: 0.3752]


Training Epoch 10: 100%|████████████████████| 4016/4016 [26:45<00:00,  2.50it/s]


Epoch 9 finished
Training completed ✅


In [17]:
# 1. Load UNet
unet = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5", 
    subfolder="unet"
).to(device)

# 2. Load CLIP text encoder
clip_text_encoder = CLIPTextModel.from_pretrained(
    "openai/clip-vit-large-patch14"
).to(device)
clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

projection = Projection().to(device)  # Use Projection class definition

# Load the checkpoint
checkpoint = torch.load("final_model_checkpoint.pt", map_location=device)
vae = pipeline.vae.to(device)
scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)

unet.load_state_dict(checkpoint['unet'])
clip_text_encoder.load_state_dict(checkpoint['clip_text_encoder'])
projection.load_state_dict(checkpoint['projection'])

# Set models to evaluation mode
unet.eval()
clip_text_encoder.eval()
projection.eval()

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/var/folders/9l/8k8d_tn97jdgsjynhl5q7yr80000gn/T/ipykernel_98781/3514179415.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the 

Projection(
  (proj): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): Tanh()
  )
)

### EVALUATION 


In [30]:
print(f"UNet dtype: {next(unet.parameters()).dtype}")
print(f"CLIP dtype: {next(clip_text_encoder.parameters()).dtype}")
print(f"Projection dtype: {next(projection.parameters()).dtype}")
print(f"VAE dtype: {next(vae.parameters()).dtype}")

UNet dtype: torch.float32
CLIP dtype: torch.float32
Projection dtype: torch.float32
VAE dtype: torch.float16


In [40]:
from torch.utils.data import Subset
subset_loader = torch.utils.data.DataLoader(
    Subset(test_dataset, range(500)),  
    batch_size=8,
    shuffle=False,
)

In [ ]:
evaluator = Evaluator(device)
evaluator.reset()

# === Determine VAE dtype and latent shape ===
with torch.no_grad():
    sample_batch = next(iter(subset_loader))["image"].to(device)
    vae_dtype = next(vae.parameters()).dtype
    latent = vae.encode(sample_batch.to(dtype=vae_dtype)).latent_dist.sample() * 0.18215
    latent_shape = latent.shape[1:]

# === Evaluation loop ===
with torch.no_grad():
    for batch in tqdm(subset_loader, desc="Evaluating"):
        images = batch["image"].to(device, dtype=vae_dtype)
        texts = batch["caption_text"]

        # === Encode text ===
        inputs = clip_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
        text_feats = clip_text_encoder(**inputs).last_hidden_state
        pooled_feats = text_feats.mean(dim=1)
        projected_feats = projection(pooled_feats).to(device=device, dtype=vae_dtype).unsqueeze(1)

        # === Prepare scheduler & latents ===
        latents = torch.randn((images.size(0), *latent_shape), dtype=torch.float32, device=device)
        scheduler.set_timesteps(20)
        latents = latents * scheduler.init_noise_sigma

        # === Denoising loop ===
        for t in scheduler.timesteps:
            t_tensor = torch.tensor([t], device=device).long().expand(images.size(0))
            latents = latents.float()  # ensure float32
            noise_pred = unet(latents, t_tensor, encoder_hidden_states=projected_feats.float()).sample
            latents = scheduler.step(model_output=noise_pred, timestep=t, sample=latents).prev_sample

        # === Decode latents ===
        pred_latents = latents.to(dtype=vae_dtype)
        fake_images = vae.decode(pred_latents / 0.18215).sample
        fake_images = torch.clamp((fake_images + 1.0) / 2.0, 0, 1)

        # === Compute metrics ===
        evaluator.compute_metrics(images, fake_images, texts)

# === Print final scores ===
metrics = evaluator.get_metrics()
print(f"FID: {metrics['fid']:.2f}")
print(f"Inception Score: {metrics['inception_score']:.2f}")
print(f"CLIP Score: {metrics['clip_score']:.2f}")

In [ ]:
# === Print final scores ===
metrics =  evaluator.compute_metrics(images, fake_images, texts)
print(f"FID: {metrics['fid']:.2f}")
print(f"Inception Score: {metrics['inception_score']:.2f}")
print(f"CLIP Score: {metrics['clip_score']:.2f}")

In [ ]:
import numpy as np
# === Determine VAE dtype and latent shape ===
with torch.no_grad():
    sample_batch = next(iter(subset_loader))["image"].to(device)
    vae_dtype = next(vae.parameters()).dtype
    latent = vae.encode(sample_batch.to(dtype=vae_dtype)).latent_dist.sample() * 0.18215
    latent_shape = latent.shape[1:]

# Custom text for generation
custom_text = "man standing"  # Change this to your desired text

# === Prepare text input ===
with torch.no_grad():
    inputs = clip_tokenizer([custom_text], return_tensors="pt", padding=True, truncation=True).to(device)
    text_feats = clip_text_encoder(**inputs).last_hidden_state
    pooled_feats = text_feats.mean(dim=1)
    projected_feats = projection(pooled_feats).to(device=device, dtype=vae_dtype).unsqueeze(1)

    # === Prepare scheduler & latents ===
    latents = torch.randn((1, *latent_shape), dtype=torch.float32, device=device)
    scheduler.set_timesteps(20)
    latents = latents * scheduler.init_noise_sigma

    # === Denoising loop ===
    for t in scheduler.timesteps:
        t_tensor = torch.tensor([t], device=device).long().expand(1)
        latents = latents.float()  # ensure float32
        
        # Scale model input if scheduler requires it
        if hasattr(scheduler, 'scale_model_input'):
            latents = scheduler.scale_model_input(latents, t)
            
        noise_pred = unet(latents, t_tensor, encoder_hidden_states=projected_feats.float()).sample
        latents = scheduler.step(noise_pred, t, latents).prev_sample

    # === Decode latents ===
    pred_latents = latents.to(dtype=vae_dtype)
    fake_image = vae.decode(pred_latents / 0.18215).sample
    fake_image = torch.clamp((fake_image + 1.0) / 2.0, 0, 1)

# === Convert image for display ===
# Move to CPU and convert to numpy
image_np = fake_image.squeeze(0).permute(1, 2, 0).cpu().numpy()

# Ensure proper dtype and value range
if image_np.dtype != np.float32:
    image_np = image_np.astype(np.float32)
image_np = np.clip(image_np, 0, 1)  # Ensure values are between 0 and 1

# === Plot the generated image ===
plt.figure(figsize=(8, 8))
plt.imshow(image_np)
plt.title(f'Generated Image for: "{custom_text}"')
plt.axis('off')

# Save the figure to avoid display issues
plt.savefig('generated_image.png', bbox_inches='tight', pad_inches=0)
plt.close()

# Display the saved image (alternative method)
from IPython.display import Image, display
display(Image(filename='generated_image.png'))

In [ ]:
# Check if MPS is available and set the device accordingly
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Load pre-trained stable diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe.to(device)

# Generate an image from a text prompt
prompt = "A futuristic cityscape at sunset"
image = pipe(prompt).images[0]

# Save or display the image
image.save("generated_image.png")
image.show()